<a href="https://colab.research.google.com/github/Josuerc0213/detecci-n-se-ales-de-tr-nsito-verticales/blob/main/Deteccion_se%C3%B1ales_transito_verticales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
!pip install split-folders pillow ultralytics
!pip install roboflow

In [ ]:



from roboflow import Roboflow
rf = Roboflow(api_key="qYipyvnCKi0WYlO8X8wx")
project = rf.workspace("seales-transito").project("senales_de_transito_verticales_deteccion-2-modi")
version = project.version(8)
dataset = version.download("yolov11")






In [ ]:
# clases minoritarias
minority_classes = ["NO VIRAR EN U","NO VIRAR IZQUIERDA O DERECHA", "CICLOVIA", "PEATONES EN LA VIA", "CEDA PASO",  "DOBLE VIA" ]  # ✅ Múltiples clases
n_copias = 5  # Número de veces que se duplicarán las imágenes

# Duplicar imágenes y etiquetas
import os, shutil


images_dir = os.path.join(dataset.location, "valid/images")
labels_dir = os.path.join(dataset.location, "valid/labels")

for clase in minority_classes:
    for i in range(n_copias):
        for filename in os.listdir(images_dir):
            if clase in filename:
                src_img = os.path.join(images_dir, filename)
                dst_img = os.path.join(images_dir, f"copy{i}_{clase}_{filename}")
                shutil.copy(src_img, dst_img)


                label_file = filename.replace(".jpg", ".txt").replace(".png", ".txt")
                src_lbl = os.path.join(labels_dir, label_file)
                dst_lbl = os.path.join(labels_dir, f"copy{i}_{clase}_{label_file}")
                if os.path.exists(src_lbl):
                    shutil.copy(src_lbl, dst_lbl)

In [ ]:
# Specify the path to the file
file_path = '/content/Señales_de_tránsito_verticales_detección-2-modi-8/data.yaml'

# Check if the file exists
if os.path.exists(file_path):
    print(f"The file at {file_path} exists.")
else:
    print(f"The file at {file_path} does not exist.")

In [ ]:
# Get the current working directory
cwd = os.getcwd()
print(f"Current working directory: {cwd}")

# List all files and directories in the current directory
files = os.listdir(cwd)
print(f"Files and directories in '{cwd}': {files}")


In [ ]:
dir_path = cwd + "/sample_data"
for files in os.listdir(dir_path):
  os.remove(dir_path + "/" + files)
os.rmdir(dir_path)

In [ ]:
from ultralytics import YOLO
model = YOLO("yolo11n.pt")
model.info()

In [ ]:
from ultralytics import settings

# View all settings

settings.update({'datasets_dir': '/content/Señales_de_tránsito_verticales_detección-2-modi-8'})

In [ ]:
# os.chdir("/content")
# Entrenar el modelo
model.train(
    data='/content/Señales_de_tránsito_verticales_detección-2-modi-8/data.yaml',
    epochs=50,
    imgsz=640,
    batch=16,  # Ajusta según tu GPU
    name="entrenamiento_señales_de_transito",
    project="proyecto_señales_de_transito"
)

In [ ]:
# Cargar el modelo entrenado
# modelo_entrenado = YOLO("/content/yolov8n.pt")
modelo_entrenado = YOLO("")

# Realizar inferencia en una imagen
resultados = modelo_entrenado.predict(source="", save=True)

# Acceder a los resultados
for resultado in resultados:
    print(f"Predicciones para la imagen: {resultado.path}")

    for caja, conf, clase in zip(resultado.boxes.xyxy, resultado.boxes.conf, resultado.boxes.cls):
        print(f"Bounding Box: {caja} - Confianza: {conf} - Clase: {clase}")

In [ ]:
# Cargar y mostrar la imagen con las predicciones
img = Image.open("")
plt.imshow(img)
plt.axis('off')  # Ocultar los ejes
plt.show()

In [ ]:
import shutil
from google.colab import files

# Especifica el nombre de la carpeta que deseas comprimir y descargar
folder_path = "/content/proyecto_señales_de_transito/entrenamiento_señales_de_transito"  # Ruta de la carpeta en Colab
output_filename = "señales_ctipo.zip"   # Nombre del archivo zip de salida

# Comprimir la carpeta en un archivo ZIP
shutil.make_archive(output_filename.replace('.zip', ''), 'zip', folder_path)

# Descargar el archivo ZIP
files.download(output_filename)
